# SageMaker PySpark XGBoost MNIST

1. [Введение](#Введение)
2. [Установка](#Установка)
3. [Загрузка данных](#Загрузка-данных)
4. [Тренировка и хостинг модели](#Тренировка-и-хостинг-модели)
5. [Инференс](#Инференс)

## Введение

Этот ноутбук покажет как классифицировать рукописные цифры из датасета MNIST с использованием алгоритма XGboost на Amazon Sagemaker с использованием библиотеки SageMaker PySpark. После тренировки мы будем хостить модель на Amazon Sagemaker и получать предсказания с помощью нее.

Больше информации о Sagemaker Spark можно найти по ссылке на GitHub: https://github.com/aws/sagemaker-spark

Также можно посетить репозиторий https://github.com/dmlc/xgboost для того, чтобы узнать больше о XGboost алгоритме.

## Установка

Для начала, мы импортируем необходимые библиотеки и создаем SparkSession() с использованием зависимостей SageMaker Spark.

In [ ]:
import os

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Конфигурация для использования SageMaker Spark зависимостей
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# sagemaker-pyspark-sdk секция репозитория SageMaker Spark
# покажет как коннектиться к удаленному EMR кластеру 
spark = SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .master("local[*]").getOrCreate()

## Загрузка данных

libsvm формат MNIST датасета, который мы будем использовать для тренировки нашей модели доступен в s3-bucket:

`s3://sagemaker-sample-data-[region]/spark/mnist/train/`

где [region] меняется на регион aws, в котором развернуты и работают ваши сервисы.

Больше информации о формате libsvm, использующемся в данном ноутбуке, можно найти по адресу: https://spark.apache.org/docs/2.0.2/mllib-data-types.html

Информация о коннекте SageMaker Notebook Instance к удаленному EMR кластеру можно найти по ссылке: https://aws.amazon.com/blogs/machine-learning/build-amazon-sagemaker-notebooks-backed-by-spark-in-amazon-emr/

In [ ]:
import boto3

cn_regions = ['cn-north-1', 'cn-northwest-1']
region = boto3.Session().region_name
endpoint_domain = 'com.cn' if region in cn_regions else 'com'
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 's3.{}.amazonaws.{}'.format(region, endpoint_domain))

trainingData = spark.read.format('libsvm')\
    .option('numFeatures', '784')\
    .option('vectorType', 'dense')\
    .load('s3a://sagemaker-sample-data-{}/spark/mnist/train/'.format(region))

testData = spark.read.format('libsvm')\
    .option('numFeatures', '784')\
    .option('vectorType', 'dense')\
    .load('s3a://sagemaker-sample-data-{}/spark/mnist/test/'.format(region))

trainingData.show()

## Тренировка и хостинг модели

Сейчас мы создадим XGBoostSageMakerEstimator, который использует XGBoost SageMaker алгоритм для тренировки наших входящих данных и использует SageMaker docker image для хостинга нашей модели.

Вызывая метод fit(), мы обучим нашу модель на SageMaker и затем создадим endpoint для хостинга. 

Данный код запускает тренировку и создает endpoint. Данный процесс занимает около 15-20 минут:

In [ ]:
import random
from sagemaker_pyspark import IAMRole, S3DataPath
from sagemaker_pyspark.algorithms import XGBoostSageMakerEstimator

xgboost_estimator = XGBoostSageMakerEstimator(
    sagemakerRole=IAMRole(role),
    trainingInstanceType='ml.m4.xlarge',
    trainingInstanceCount=1,
    endpointInstanceType='ml.m4.xlarge',
    endpointInitialInstanceCount=1)

xgboost_estimator.setEta(0.2)
xgboost_estimator.setGamma(4)
xgboost_estimator.setMinChildWeight(6)
xgboost_estimator.setSilent(0)
xgboost_estimator.setObjective("multi:softmax")
xgboost_estimator.setNumClasses(10)
xgboost_estimator.setNumRound(10)

# тренировка модели
model = xgboost_estimator.fit(trainingData)

## Инференс

Теперь мы трансформим наш датафрейм.

Для того, чтобы сделать это, мы сериализируем каждую строчку "фичей" векторов в libsvm формат для выполнения инференс задач в Sagemaker endpoint. Полученный csv ответ от XGboost мы десериализируем и помещаем в наш датафрейм. Эта сериализация/десериализация происходит в методе transform():

In [ ]:
transformedData = model.transform(testData)

transformedData.show()

Как хорошо отработал алгоритм? Нарисуем цифры, относящиеся к каждому классу и вручную оценим полученные результаты:

In [ ]:
from pyspark.sql.types import DoubleType
import matplotlib.pyplot as plt
import numpy as np

# helper функция для отрисовки цифр
def show_digit(img, caption='', xlabel='', subplot=None):
    if subplot==None:
        _,(subplot)=plt.subplots(1,1)
    imgr=img.reshape((28,28))
    subplot.axes.get_xaxis().set_ticks([])
    subplot.axes.get_yaxis().set_ticks([])
    plt.title(caption)
    plt.xlabel(xlabel)
    subplot.imshow(imgr, cmap='gray')

images = np.array(transformedData.select("features").cache().take(250))
clusters = transformedData.select("prediction").cache().take(250)

for cluster in range(10):
    print('\n\n\nCluster {}:'.format(int(cluster)))
    digits=[ img for l, img in zip(clusters, images) if int(l.prediction) == cluster ]
    height=((len(digits) - 1) // 5) + 1
    width=5
    plt.rcParams["figure.figsize"] = (width,height)
    _, subplots = plt.subplots(height, width)
    subplots=np.ndarray.flatten(subplots)
    for subplot, image in zip(subplots, digits):
        show_digit(image, subplot=subplot)
    for subplot in subplots[len(digits):]:
        subplot.axis('off')

    plt.show()

После успешного выполнения задачи, endpoint может быть удален:

In [ ]:
# Удаление endpoint

from sagemaker_pyspark import SageMakerResourceCleanup

resource_cleanup = SageMakerResourceCleanup(model.sagemakerClient)
resource_cleanup.deleteResources(model.getCreatedResources())